# Answering Business Questions using SQL


### Introduction and Schema Diagram

In this guided project, we're going to practice using our SQL skills to answer business questions.

We'll use Chinook database which provided as a SQLite database file called `chinook.db`. A copy of the database schema is below - we need to come back and check the connection between each other very often.

![connection](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)



### Connect to database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Overview of the Data

write a query to return information of the tables.

In [2]:
%%sql
SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ('table','view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:


|Artist Name|Genre|
|---|---|
|Regal|Hip-Hop|
|Red Tone|Punk|
|Metero and the Girls|Pop|
|Slim Jim Bites|Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out <b>which genres sell the best in the USA</b>.

We need to write a query to find out which genres sell the most tracks in the USA, write up a summary of findings, and make a recommendation for the three artists whose albums we should purchase for the store.



### Which genres sell the most tracks in the USA?

In [3]:
%%sql
WITH usa_track_sold AS
    (
    SELECT il.*
    FROM invoice_line 'il'
    INNER JOIN invoice 'inv' ON il.invoice_id=inv.invoice_id
    INNER JOIN customer 'c' ON c.customer_id=inv.customer_id
    WHERE c.country='USA'
    )

SELECT
    g.name 'genre',
    COUNT(uts.invoice_line_id) 'quantity',
    ROUND(CAST(COUNT(uts.invoice_line_id) as Float)/
    (SELECT COUNT(*) from usa_track_sold)*100,2)||'%' 'percentage'
FROM usa_track_sold 'uts'
INNER JOIN track 't' ON t.track_id=uts.track_id
INNER JOIN genre 'g' ON g.genre_id=t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,quantity,percentage
Rock,561,53.38%
Alternative & Punk,130,12.37%
Metal,124,11.8%
R&B/Soul,53,5.04%
Blues,36,3.43%
Alternative,35,3.33%
Latin,22,2.09%
Pop,22,2.09%
Hip Hop/Rap,20,1.9%
Jazz,14,1.33%


The best selling genre in USA is the following:
* Rock(561 times/ 53.38%)
* Alternative & Punk (130 times/ 12.37%)
* Metal(124 times/11.8%)

### Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to <b>analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others</b>.

We might like to consider whether any extra columns from the employee table explain any variance we see, or whether the variance might instead be indicative of employee performance.

In [5]:
%%sql
SELECT
    e.first_name||" "||e.last_name employee_name,
    e.hire_date,
    e.birthdate,
    e.country,
    e.title title,
    SUM(inv.total) total_dollar_amount
FROM employee e
INNER JOIN customer c ON e.employee_id=c.support_rep_id
INNER JOIN invoice inv ON c.customer_id=inv.customer_id
GROUP BY 1,4
ORDER BY 6 DESC;

Done.


employee_name,hire_date,birthdate,country,title,total_dollar_amount
Jane Peacock,2017-04-01 00:00:00,1973-08-29 00:00:00,Canada,Sales Support Agent,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1947-09-19 00:00:00,Canada,Sales Support Agent,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1965-03-03 00:00:00,Canada,Sales Support Agent,1393.920000000002


We connect the database of employee, customer and invoice to find out total dollar amount of sales to each sales support agent.

We filter employee title to focus on `sales support agent`. Then, review them in descending order to find out Jane Peacock is responsible for most dollar, while Steve Johnson is the least.

When we look deeper in the hire date, we find out there's positive connection between hire date and total dollar amount. The earlier sales support agent was hired, the more dollar amount it is. This makes sense because employee will get more familiar with selling process and had better chance to complete sales as working for longer time.

All of them are Canadian so we can't tell if there's relationship between which country they came from. According to age, I can't tell if age can affect the total dollar amount because there's no obvious relationship for birthdate.

### Analyze the sales data for customers from each different country

Our next task is to analyze the sales data for customers from each different country. We've been given guidance to use the country value from the `customers` table, and ignore the country from the billing address in the `invoice` table.

In particular, you have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

There're some countries with only one customer, we should group these customers as "Others" first.

In [9]:
%%sql
WITH country_or_other AS
    (
    SELECT 
        CASE 
            WHEN (SELECT COUNT(*) 
                  FROM customer
                  WHERE country=c.country
                 ) = 1 THEN 'Other'
            ELSE c.country
        END AS 'country',
        c.customer_id,
        il.*
    FROM invoice_line 'il'
    INNER JOIN invoice 'i' ON i.invoice_id=il.invoice_id
    INNER JOIN customer 'c' ON i.customer_id=c.customer_id
    )
    
SELECT 
    country,
    number_of_customer,
    total_sales,
    avg_sales_per_person,
    avg_order_sales
    
FROM
    (
    SELECT
        country,
        COUNT(DISTINCT customer_id) 'number_of_customer',
        ROUND(SUM(unit_price),2) 'total_sales',
        ROUND(SUM(unit_price)/COUNT(DISTINCT customer_id),2) 'avg_sales_per_person',
        ROUND(SUM(unit_price)/COUNT(DISTINCT invoice_id),2) 'avg_order_sales',
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS 'sort'
    FROM country_or_other
    GROUP BY country
    ORDER BY sort, total_sales DESC
    );

Done.


country,number_of_customer,total_sales,avg_sales_per_person,avg_order_sales
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


First, we calculat the number of customers by country and find many countries only have 1 customers. We'd like to group them into `Other` so the result will be tidy.

Second, we finish calculations in subqueries. Remember to sort data by total sales in descending way, with 'Other' group at the very buttom.

Finally, sort the result by the total sales in descending way, with the 'Other' group at the very bottom.
(use the technique `CASE` to assign 'Other' country 1, other countries 0)

### Albums vs. Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in the following ways:
- Purchase a whole album
- Purchase a collection of one or more individual tracks

When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.


<b>We have been asked to find out what percentage of purchases are individual tracks vs whole albums</b>, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when we are performing an analysis to have 'edge cases' which prevent us from getting a 100% accurate answer to our question. In this instance, we have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In [10]:
%%sql

WITH invoice_first_track AS
    (
    SELECT
        il.invoice_id 'invoice_id',
        MIN(il.track_id) 'first_track_id'
    FROM invoice_line 'il'
    GROUP BY 1
    )
    
SELECT
    album_purchase,
    COUNT(invoice_id) 'number_of_invoices',
    ROUND(CAST(COUNT(invoice_id)as Float)/(SELECT COUNT(*) FROM invoice)*100,2)||'%' 'percentage_of_invoice'
FROM (
    SELECT 
        ift.*,
        CASE
            WHEN
                (
                SELECT t.track_id 
                FROM track t
                WHERE t.album_id = (
                                    SELECT t2.album_id 
                                    FROM track t2
                                    WHERE t2.track_id = ift.first_track_id
                                    )
                EXCEPT
                    
                SELECT il2.track_id 
                FROM invoice_line 'il2'
                WHERE il2.invoice_id = ift.invoice_id
                )IS NULL
            AND
                (
                SELECT il2.track_id 
                FROM invoice_line 'il2'
                WHERE il2.invoice_id = ift.invoice_id
                
                EXCEPT
                    
                SELECT t.track_id 
                FROM track t
                WHERE t.album_id = (
                                    SELECT t2.album_id 
                                    FROM track 't2'
                                    WHERE t2.track_id = ift.first_track_id
                                    )
                )IS NULL
        THEN 'YES'
        ELSE 'NO'
        END AS 'album_purchase'
     FROM invoice_first_track 'ift'               
    )
GROUP BY album_purchase;  


Done.


album_purchase,number_of_invoices,percentage_of_invoice
NO,500,81.43%
YES,114,18.57%


Following observations above, we'd like to recommend Chinook only to purchase the most popular tracks instead of a whold album to save budget because only 18.57% customers will buy a whole album.

### Next Steps

##### Which artist is used in the most playlist?

In [13]:
%%sql
SELECT
    a.name 'artist',
    COUNT(pl.playlist_id) 'number_of_times',
    ROUND(CAST(COUNT(pl.playlist_id)as float)/(SELECT COUNT(*) FROM playlist),2)||'%' 'percentage',
    pl.name 'playlist',
    g.name 'genre'
FROM artist 'a'
INNER JOIN album 'al' ON a.artist_id=al.artist_id
INNER JOIN track 't' ON al.album_id=t.album_id
INNER JOIN playlist_track 'plt' ON t.track_id=plt.track_id
INNER JOIN playlist 'pl' ON pl.playlist_id=plt.playlist_id
INNER JOIN genre 'g' ON g.genre_id=t.genre_id
GROUP BY artist
ORDER BY number_of_times DESC
LIMIT 10;

Done.


artist,number_of_times,percentage,playlist,genre
Iron Maiden,516,28.67%,Heavy Metal Classic,Metal
U2,333,18.5%,Music,Pop
Metallica,296,16.44%,Heavy Metal Classic,Metal
Led Zeppelin,252,14.0%,Music,Rock
Deep Purple,226,12.56%,Music,Rock
Lost,184,10.22%,TV Shows,Drama
Pearl Jam,177,9.83%,Grunge,Rock
Eric Clapton,145,8.06%,Brazilian Music,Latin
Faith No More,145,8.06%,Music,Alternative & Punk
Lenny Kravitz,143,7.94%,Music,Metal


`Iron Maiden` is the artist most used in the playlist for 516 times and 28.6%, followed by U2 (333 times,18.5%) and Metallica (296 times, 16.44%).

##### How many tracks have been purchased vs not purchased?

In [14]:
%%sql
WITH invoiced AS 
    (
    SELECT track_id
    FROM invoice_line
    )

SELECT COUNT(DISTINCT i.track_id) 'tracks_purchased',
       (
       SELECT COUNT(DISTINCT t.track_id)
       FROM track 't'
       WHERE t.track_id NOT IN (SELECT track_id FROM invoiced)
       ) AS 'tracks_not_purchased'
FROM invoiced 'i'
LEFT JOIN track 't' ON t.track_id = i.track_id;


Done.


tracks_purchased,tracks_not_purchased
1806,1697


1806 tracks have been purchased, while 1697 tracks haven't been purchased.

##### Do protected vs non-protected media types have an effect on popularity?

In [15]:
%%sql
SELECT *
FROM media_type;

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


As we can figure out in the context, 2 & 3 are media types which are protected. 

Let's find out if the protection will affect popularity.

In [26]:
%%sql
WITH protected AS
(
    SELECT SUM(il.unit_price * il.quantity) 'total_price',
           il.track_id,
           t.media_type_id
    FROM invoice_line 'il'
    INNER JOIN track 't' ON t.track_id = il.track_id
    WHERE t.media_type_id = 2 or t.media_type_id = 3
),
unprotected AS
(
    SELECT SUM(il.unit_price * il.quantity) 'total_price',
           il.track_id,
           t.media_type_id
    FROM invoice_line 'il'
    INNER JOIN track 't' ON t.track_id = il.track_id
    WHERE t.media_type_id != 2 and t.media_type_id != 3
)

SELECT ROUND(SUM(p.total_price),2) 'Protected',
       ROUND(SUM(u.total_price),2) 'Unprotected'
FROM media_type 'm'
LEFT JOIN protected 'p' ON p.media_type_id = m.media_type_id
LEFT JOIN unprotected 'u' ON u.media_type_id = m.media_type_id;

Done.


Protected,Unprotected
437.58,4271.85


Total sales price of unprotected media type is 4271.85 which is quite larger than that of protected media type of 437.58.

We can make a conclusion that based on this database, unprotected media types are more popular than protected ones.

![connection](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)